In [54]:
# import splinter and BeatifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from quant_up import user, password
import time 
import datetime
import pandas as pd

In [55]:
### Navigate to quantum online,input credentials and naviagte IPOS of income securities page

In [56]:
## confirm todays date 
datetime.datetime.today().strftime("%Y-%m-%d")

'2023-03-16'

In [103]:
browser = Browser('chrome')

In [104]:
# Visit quantum
url = 'https://www.quantumonline.com/listwipo.cfm?type=IncomeIPOs&RequestTimeout=90'
browser.visit(url)

In [105]:
# path to login click
browser.find_by_xpath('/html/body/table[2]/tbody/tr/td[2]/font[2]/p[1]/b/a').click()


In [106]:
# path to login click
browser.find_by_xpath('/html/body/table/tbody/tr[2]/td/table/tbody/tr[3]/td[9]/a/img').click()


In [107]:
#input user and pass

browser.find_by_xpath('/html/body/table[2]/tbody/tr/td[2]/font/font/blockquote/center/font/table/tbody/tr[2]/td/input').fill(user)
                      
browser.find_by_xpath('/html/body/table[2]/tbody/tr/td[2]/font/font/blockquote/center/font/table/tbody/tr[3]/td/input').fill(password)

# click
browser.find_by_xpath('/html/body/table[2]/tbody/tr/td[2]/font/font/blockquote/center/font/table/tbody/tr[4]/td/input').click()



In [108]:
#path to special lists
browser.find_by_xpath('/html/body/table[1]/tbody/tr[2]/td/table/tbody/tr[3]/td[6]/a/img').click()

In [109]:
#path to ipos
browser.find_by_xpath('/html/body/font/table/tbody/tr/td[2]/font[2]/p[2]/b[1]/a').click()
time.sleep(5)

In [64]:
### SCRAPE

In [65]:
#scrape
html=browser.html
ipo_soup = soup(html, 'html.parser')

In [66]:
# Create empty lists for the table headers and rows
header_list = []
row_list = []

# Find all table rows
tables = ipo_soup.find_all('tr')

# Loop through the table rows
for i, row in enumerate(tables):
    # Find all table cells (td) and table headers (th) within the row
    cells = row.find_all(['td', 'th'])
    # If this is the first row, treat it as the header row
    if i == 0:
        # Extract the text from each cell and append to the header list
        header_list = [cell.text.strip() for cell in cells]
    else:
        # Extract the text from each cell and append to the row list
        row_data = [cell.text.strip() for cell in cells]
        row_list.append(row_data)

# Print the header and row lists
print(header_list)
print(row_list)


['Quick Search\n\n\n\n\nby Ticker Symbol\nby CUSIP Number\nSymbol Lookup', 'Quick Search', '', 'by Ticker Symbol\nby CUSIP Number\nSymbol Lookup', '', 'Welcome, \xa0\xa0\xa0Sarah Peterson \n\t\t\t,\xa0\xa0\xa0  Non-Contributor (Please click here to make a contribution) \n\t\t\t\n\t\t\tDays of use: 7', '348304', '']
[['Quick Search'], [''], ['by Ticker Symbol\nby CUSIP Number\nSymbol Lookup'], [''], [''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', ''], ['', ''], ['', '', '', '', '', '', '', '', '', '', ''], ['', ''], ['', ''], ['IPOs of Income Securities'], ['A list of 946 Initial Public Offerings (IPOs) of income and related securities.  The securities included on the list include all income securities that are included on the tables shown on the Income Tables menu. The securities are listed in order by IPO date with the most recent IPOs listed first.  New IPOs appear on 

In [67]:
### Grab relevant info from scrape and convert to pandas DataFrame

In [134]:
# Find the starting index for 'SymbolCUSIP' in row_list
start_index = None
for i, row in enumerate(row_list):
    if 'SymbolCUSIP' in row:
        start_index = i
        break

# Remove all rows before the 'SymbolCUSIP' row (including the header row)
data = row_list[start_index+1:]

# Create a pandas DataFrame from the remaining rows
IPO_df = pd.DataFrame(data)
IPO_df=IPO_df.loc[:,[0,1,2,3,4]]
IPO_df.columns=['SymbolCUSIP', 'Security', 'IPO Date', 'Stock Exchange', 'Security Type']
IPO_df=IPO_df.loc[IPO_df['SymbolCUSIP']!= 'SymbolCUSIP']
IPO_df.head()

,SymbolCUSIP,Security,IPO Date,Stock Exchange,Security Type
0,FTAIMG3730V147,FTAI Aviation Ltd Reset Rate Cumulative Series...,IPO: 3/08/2023 \n\n\n\t IPO Prospectus @ S...,NGS\nChart,Pfd Stock
1,JXNFV46817M206,Jackson Financial Inc Dep Shares Reset Rate Se...,IPO: 3/06/2023 \n\n\n\t IPO Prospectus @ S...,OTOTC\nChart,Pfd Stock
2,HBANL446150773,Huntington Bancshares Dep Shares Series J Rese...,IPO: 2/27/2023 \n\n\n\t IPO Prospectus @ S...,NGS\nChart,Pfd Stock
3,ASBA045487600,Associated Banc-Corp Reset Rate Subordinated N...,IPO: 2/06/2023 \n\n\n\t IPO Prospectus @ S...,NYSE\nChart,E/T Debt Sec.
4,EFC-C28852N406,Ellington Financial Inc. Series C Reset Rate C...,IPO: 2/02/2023 \n\n\n\t IPO Prospectus @ S...,NYSE\nChart,Pfd Stock


In [135]:
### Data cleaning (split IPO date into 2 columns: ipo date and call date, 
## move IPO Date so that it is next to call date)
# ensure the tail end of the data is 

In [136]:
IPO_df.drop(IPO_df.tail(7).index, inplace = True)

In [137]:
IPO_df[['IPO Date Two', 1,2,3,4, 'Call Date', 6,7,8,9,10]]=IPO_df["IPO Date"].str.split(" ", expand=True)

In [138]:
IPO_df=IPO_df.drop([1,2,3,4,6,7,8,9,10], axis=1)

In [139]:
IPO_df["IPO Date Two"]= IPO_df["IPO Date Two"].convert_dtypes()

In [140]:
IPO_df["IPO Date"]=IPO_df["IPO Date Two"].str.slice(4,15)

In [141]:
IPO_df["Call Date"]= IPO_df["Call Date"].convert_dtypes()

In [142]:
IPO_df["Call Date"]=IPO_df["Call Date"].str.slice(6,16)

In [143]:
del IPO_df["IPO Date Two"]

In [144]:
IPO_df = IPO_df[['SymbolCUSIP', 'Security', 'IPO Date', 'Call Date', 'Stock Exchange', 'Security Type']]

In [145]:
IPO_df.dtypes

SymbolCUSIP       object
Security          object
IPO Date          string
Call Date         string
Stock Exchange    object
Security Type     object
dtype: object

In [146]:
IPO_df.tail(10)

,SymbolCUSIP,Security,IPO Date,Call Date,Stock Exchange,Security Type
93,VNO-O929042794,Vornado Realty Trust 4.45% Series O Cumulative...,9/13/2021,9/22/2026,NYSE\nChart,Pfd Stock
94,RITM-D64828T706,Rithm Capital Corp. Reset Rate Series D Cumula...,9/13/2021,11/15/2026,NYSE\nChart,Pfd Stock
95,ADC-A008492209,Agree Realty Corp 4.250% Depositary Shares Ser...,9/08/2021,9/17/2026,NYSE\nChart,Pfd Stock
96,CHRB15957P204,"Charah Solutions, Inc. 8.50% Senior Notes Due ...",9/01/2021,8/31/2023,NYSE\nChart,E/T Debt Sec.
97,SLNHP583543202,"Soluna Holdings, Inc. 9.0% Series A Cumulative...",8/19/2021,8/23/2026,NCM\nChart,Pfd Stock
98,PMT-C70931T509,"PennyMac Mortgage Investment Trust, 6.75% Seri...",8/17/2021,8/24/2026,NYSE\nChart,Pfd Stock
99,TGH-B88314W303,Textainer Group Holdings Ltd 6.25% Series B Cu...,8/16/2021,12/15/2026,NYSE\nChart,Pfd Stock
100,CNOBP20786W503,"ConnectOne Bancorp, Inc. Dep Shares Reset Rate...",8/13/2021,9/01/2026,NGS\nChart,Pfd Stock
101,GAINZ376546883,Gladstone Investment Corp. 4.875% Notes Due 11...,8/12/2021,11/01/2023,NGS\nChart,E/T Debt Sec.
102,TRTN-EG9078F156,Triton International Ltd. 5.75% Ser E Cumul Re...,8/11/2021,9/15/2026,NYSE\nChart,Pfd Stock


In [147]:
# creating the date for yesterday and using it as a variable
yesterdays_date=str((datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d"))

# if you already have a date from yesterday then do this
IPO_yesterday_df= pd.read_csv("iposOfIncomeSecurities_"+yesterdays_date+".csv")

#if you want a specific date then do this
#IPO_yesterday_df= pd.read_csv("iposOfIncomeSecurities_2023-03-09.csv")
IPO_yesterday_df.head()


,SymbolCUSIP,Security,IPO Date,Stock Exchange,Security Type,Call Date
0,FTVTVG3730V147,FTAI Aviation Ltd Reset Rate Cumulative Series...,3/08/2023,OTOTC\nChart,Pfd Stock,6/15/28
1,JXNFV46817M206,Jackson Financial Inc Dep Shares Reset Rate Se...,3/06/2023,OTOTC\nChart,Pfd Stock,3/30/28
2,HBANL446150773,Huntington Bancshares Dep Shares Series J Rese...,2/27/2023,NGS\nChart,Pfd Stock,4/15/28
3,ASBA045487600,Associated Banc-Corp Reset Rate Subordinated N...,2/06/2023,NYSE\nChart,E/T Debt Sec.,3/1/28
4,EFC-C28852N406,Ellington Financial Inc. Series C Reset Rate C...,2/02/2023,NYSE\nChart,Pfd Stock,4/30/28


In [149]:
# checks to see if there are changes on the SymbolCUSIP

IPO_delta_df=pd.merge(IPO_df[['SymbolCUSIP']],IPO_yesterday_df[['SymbolCUSIP']],on='SymbolCUSIP', how='outer', indicator=True)
IPO_delta_df[IPO_delta_df['_merge'].isin(['left_only', 'right_only'])]

,SymbolCUSIP,_merge
0,FTAIMG3730V147,left_only
100,FTVTVG3730V147,right_only


In [151]:
# if you want to check changes on security name update SymbolCUSIP to Security ( do in sep cell)
# if nothing changes nothing shows
IPO_delta_name=pd.merge(IPO_df[['Security']],IPO_yesterday_df[['Security']],on='Security', how='outer', indicator=True)
IPO_delta_name[IPO_delta_name['_merge'].isin(['left_only', 'right_only'])]


,Security,_merge


In [100]:
IPO_df.to_csv("iposOfIncomeSecurities_"+datetime.datetime.today().strftime("%Y-%m-%d")+".csv", index=False)

In [99]:
browser.quit()